In [1]:
import os
import platform
import shutil
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

import sys
yolor_path = '/data/workspace/yuzijian/yolor'
sys.path.append(yolor_path)


from utils.google_utils import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import (
    check_img_size, non_max_suppression, apply_classifier, scale_coords, xyxy2xywh, strip_optimizer)
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

from models.models import *
from utils.datasets import *
from utils.general import *

/data/workspace/yuzijian/miniconda3/envs/yolo/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from easydict import EasyDict

opt = EasyDict()
opt.agnostic_nms = False
opt.augment = False
opt.cfg = 'cfg/yolor_p6.cfg'
opt.classes = None
opt.conf_thres = 0.5
opt.device = '0'
opt.img_size = 1280
opt.iou_thres = 0.5
opt.names = 'data/coco.names'
opt.output = 'inference/output'
opt.save_txt = False
opt.source = '/data/workspace/dataset/kitti_2d_detection/training/image_2/000001.png'
opt.update = False
opt.view_img = False
opt.weights = ['weights/yolor_p6.pt']

In [3]:
def load_classes(path):
    # Loads *.names file at 'path'
    with open(path, 'r') as f:
        names = f.read().split('\n')
    return list(filter(None, names))  # filter removes empty strings (such as last line)

In [4]:
opt.source = '/data/workspace/dataset/kitti_2d_detection/training/image_2'
opt.output = '/data/workspace/yuzijian/kitti_2d_detection/yolo_for_kitti_results/yolor/pretrained'
opt.save_txt = True

### 1、推理

In [5]:
out, source, weights, view_img, save_txt, imgsz, cfg, names = \
    opt.output, opt.source, opt.weights, opt.view_img, opt.save_txt, opt.img_size, opt.cfg, opt.names
webcam = source == '0' or source.startswith('rtsp') or source.startswith('http') or source.endswith('.txt')

# Initialize
device = select_device(opt.device)
if os.path.exists(out):
    shutil.rmtree(out)  # delete output folder
os.makedirs(out)  # make new output folder
half = device.type != 'cpu'  # half precision only supported on CUDA

# Load model
model = Darknet(cfg, imgsz).cuda()
model.load_state_dict(torch.load(weights[0], map_location=device)['model'])

model.to(device).eval()
if half:
    model.half()  # to FP16

# Second-stage classifier
classify = False
if classify:
    modelc = load_classifier(name='resnet101', n=2)  # initialize
    modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model'])  # load weights
    modelc.to(device).eval()


# Set Dataloader
vid_path, vid_writer = None, None
if webcam:
    view_img = True
    cudnn.benchmark = True  # set True to speed up constant image size inference
    dataset = LoadStreams(source, img_size=imgsz)
else:
    save_img = True
    dataset = LoadImages(source, img_size=imgsz, auto_size=64)


# Get names and colors
names = load_classes(names)
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

# Run inference
t0 = time.time()
img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
_ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
with torch.no_grad():
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=opt.augment)[0]

        # Apply NMS
        pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)
        t2 = time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0 = path[i], '%g: ' % i, im0s[i].copy()
            else:
                p, s, im0 = path, '', im0s

            save_path = str(Path(out) / Path(p).name)
            txt_path = str(Path(out) / Path(p).stem) + ('_%g' % dataset.frame if dataset.mode == 'video' else '')
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += '%g %ss, ' % (n, names[int(c)])  # add to string

                # Write results
                for *xyxy, conf, cls in det:
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * 5 + '\n') % (cls, *xywh))  # label format

                    if save_img or view_img:  # Add bbox to image
                        label = '%s %.2f' % (names[int(cls)], conf)
                        plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=3)

            # Print time (inference + NMS)
            print('%sDone. (%.3fs)' % (s, t2 - t1))

            # Stream results
            if view_img:
                cv2.imshow(p, im0)
                if cv2.waitKey(1) == ord('q'):  # q to quit
                    raise StopIteration

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'images':
                    cv2.imwrite(save_path, im0)
                else:
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer

                        fourcc = 'mp4v'  # output video codec
                        fps = vid_cap.get(cv2.CAP_PROP_FPS)
                        w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*fourcc), fps, (w, h))
                    vid_writer.write(im0)

if save_txt or save_img:
    print('Results saved to %s' % Path(out))
    if platform == 'darwin' and not opt.update:  # MacOS
        os.system('open ' + save_path)

print('Done. (%.3fs)' % (time.time() - t0))

/data/workspace/yuzijian/miniconda3/envs/yolo/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


image 1/7481 /data/workspace/dataset/kitti_2d_detection/training/image_2/000000.png: 448x1280 1 persons, 6 bicycles, Done. (0.714s)
image 2/7481 /data/workspace/dataset/kitti_2d_detection/training/image_2/000001.png: 448x1280 2 cars, 1 trains, Done. (0.050s)
image 3/7481 /data/workspace/dataset/kitti_2d_detection/training/image_2/000002.png: 448x1280 1 cars, 1 trucks, Done. (0.052s)
image 4/7481 /data/workspace/dataset/kitti_2d_detection/training/image_2/000003.png: 448x1280 2 cars, Done. (0.052s)
image 5/7481 /data/workspace/dataset/kitti_2d_detection/training/image_2/000004.png: 448x1280 4 cars, Done. (0.052s)
image 6/7481 /data/workspace/dataset/kitti_2d_detection/training/image_2/000005.png: 448x1280 1 persons, 1 traffic lights, Done. (0.052s)
image 7/7481 /data/workspace/dataset/kitti_2d_detection/training/image_2/000006.png: 448x1280 4 cars, 1 traffic lights, Done. (0.052s)
image 8/7481 /data/workspace/dataset/kitti_2d_detection/training/image_2/000007.png: 448x1280 3 cars, Done.

### 2、验证在coco上预训练的模型的精度

In [ ]:
from utils.metrics import fitness, fitness_p, fitness_r, fitness_ap50, fitness_ap, fitness_f

In [5]:
gt_labels_dir = '/data/workspace/yuzijian/kitti_2d_detection/yolo_version/labels'